In [0]:
from pyspark.sql.functions import from_json, col
from pyspark.sql.types import *

schema = StructType([
    StructField("transactionId", StringType()),
    StructField("cardNumber", StringType()),
    StructField("amount", DoubleType()),
    StructField("location", StringType()),
    StructField("timestamp", StringType()),
    StructField("userId", StringType())
])

event_hub_config = {
    "kafka.bootstrap.servers": "balusudha-kafka-namespace.servicebus.windows.net:9093",
    "subscribe": "transactions",
    "kafka.security.protocol": "SASL_SSL",
    "kafka.sasl.mechanism": "PLAIN",
    "kafka.sasl.jaas.config": """
        org.apache.kafka.common.security.plain.PlainLoginModule required
        username="$ConnectionString"
        password=""
    """
}

df_kafka = spark.readStream.format("kafka").options(**event_hub_config).load()

df_json = df_kafka.selectExpr("CAST(value AS STRING)") \
    .withColumn("data", from_json(col("value"), schema)) \
    .select("data.*")

sas_token_write = ""

spark.conf.set(f"fs.azure.sas.deltatables.balusudhastorageaccount.blob.core.windows.net",sas_token_write)

output_path=f"wasbs://deltatables@balusudhastorageaccount.blob.core.windows.net/output_files/"

df_json.writeStream \
    .format("delta") \
    .outputMode("append") \
    .option("checkpointLocation", f"wasbs://deltatables@balusudhastorageaccount.blob.core.windows.net/checkpoints") \
    .start(output_path)

print("data written to delta")

data written to delta


In [0]:
spark.conf.set("fs.azure.account.key.balusudhastorageaccount.blob.core.windows.net", sas_token_write)
df = spark.read.format("delta").load(

    "wasbs://deltatables@balusudhastorageaccount.blob.core.windows.net/output_files"
)

print("Total rows:", df.count())


Total rows: 467
